# Import Basics

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math

from numpy.linalg import norm
from datetime import datetime
from scipy.stats import skew  # for some statistics
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
import random
from random import sample 


from sklearn.preprocessing import normalize
from sklearn.neighbors import KNeighborsClassifier
from statsmodels.tsa.ar_model import AutoReg
from sklearn.metrics import mean_squared_error
from math import sqrt
from matplotlib import pyplot
from statsmodels.graphics.tsaplots import plot_acf

import matplotlib.pyplot as plt
from neural_net import NeuralNet, NeuralNet_Chain, NeuralNet_Multiple, NeuralNet_Chain_skLearn
from sklearn.neural_network import MLPRegressor

import add_features
import DataClean
from preprocess import preprocess


import os
import time
import utils
import pickle
import gzip

import warnings
warnings.filterwarnings('ignore')

# Introduction

our task is to predict the future position of the ego vehicle 3 seconds into the future $y_i \in \mathbb{R}^{60}$  given $x_i$, one second of vehicle position data for the ego vehicle and (up to) the ten nearest agents to the ego at the point in time where prediction starts

# Data Cleaning

in this part we are going to perform the following operations on the each dataset $X_i$:
<ol>
  <li>delete unwanted columns (time step, id, role, present)</li>
  <li>convert non-numerical values to numerical</li>
  <li>fill the empty columns so that each dataset has 10 elements (car, pedestrain/bicycle)</li>
</ol>
then we will perform the following operations on the each dataset $y_i$:
<ol>
  <li>not all the datasets contain 30 $(x, y)$ tuple, we are going to fill this broken datasets</li>
  <li>delete unwanted columns (time step)</li>
  <li>fill the empty columns so that each dataset has 10 elements (car, pedestrain/bicycle)</li>
</ol>

In [449]:
def clean_data_X(data):
    data = data.drop(columns = ['time step'])
    
    data = data.replace({" car": 1, " pedestrian/bicycle": 2})
    data = data.replace({" agent": 1, " others": 2})
    
    #delete empty columns
    data = data.replace({0: math.nan})
    data = data.dropna(how='all', axis=1)
    data = data.replace({math.nan: 0})

    #we want our data to have 10 elements (car/pedestrain/bicycle) per dataset
    m = int(data.columns[-1][-1])+1
    n = 10
    t = n-m
    for i in range(m):
        data = data.drop(columns = [' id' +str(i), ' present'+str(i)])

    if m != n:        
        s = np.random.choice(m, t)
        for i in range(m,n):      
            k = i-m
            data[' role'+str(i)] = data[' role'+str(s[k])]
            data[' type'+str(i)] = data[' type'+str(s[k])]
            data[' x'+str(i)] = data[' x'+str(s[k])]
            data[' y'+str(i)] = data[' y'+str(s[k])]
    
    
    return data

def get_agent(data):
    loc = int((np.where(np.array(data.iloc[0]) == ' agent')[0][0]-2)/6)
    data_new = data[[' x'+str(loc),' y'+str(loc)]].rename(columns = {' x'+str(loc):'x',' y'+str(loc):'y'})
    
    return data_new

def get_agent_x(data):
    loc = int((np.where(np.array(data.iloc[0]) == ' agent')[0][0]-2)/6)
    data_new = data[[' x'+str(loc)]].rename(columns = {' x'+str(loc):'x'})
    
    return data_new

def get_agent_y(data):
    loc = int((np.where(np.array(data.iloc[0]) == ' agent')[0][0]-2)/6)
    data_new = data[[' y'+str(loc)]].rename(columns = {' y'+str(loc):'y'})
    
    return data_new

# clean_data_y needs to be implemented
def clean_data_y(data):
    size = 30    
    if size != data.shape[0]:
        diff=size-data.shape[0]    
        x = data[' x'].values
        y = data[' y'].values
        t = data['time step'].values

        model_x = AutoReg(x, 5)
        model_y = AutoReg(y, 5)
        model_t = AutoReg(t, 5)
        predictions_x = model_x.fit().predict(start=len(x), end=len(x)+diff-1, dynamic=False)
        predictions_y = model_y.fit().predict(start=len(y), end=len(y)+diff-1, dynamic=False)
        predictions_t = model_t.fit().predict(start=len(t), end=len(t)+diff-1, dynamic=False)
        d = np.concatenate((predictions_t.reshape(-1,1),predictions_x.reshape(-1,1),predictions_y.reshape(-1,1)),axis=1)
        d = pd.DataFrame(d, columns=['time step',' x', ' y'])
        data = data.append(d)
    
    return data.drop(columns = ['time step'])

# Adding New Features

we are going to add some new features in this part namely speed direction, acceleration, turning.

In [3]:
def continuous_angle(x):
    
    last = 0
    out = []

    for angle in x:
        while angle < last-np.pi: angle += 2*np.pi
        while angle > last+np.pi: angle -= 2*np.pi
        last = angle
        out.append(angle)

    return np.array(out)

#%%

def speed_direction(data):

    for i in range(10):
        
        speed = np.zeros(11)
        sin_dir = np.zeros(11)
        cos_dir = np.zeros(11)
        
        x = data[' x%d' % i]
        y = data[' y%d' % i]
        
        speed[0] = np.sqrt((x[1]-x[0])**2+(y[1]-y[0])**2)
        direction = np.arctan2(y[1]-y[0],x[1]-x[0])
        sin_dir[0] = np.sin(direction)
        cos_dir[0] = np.cos(direction)
        
        speed[10] = np.sqrt((x[10]-x[9])**2+(y[10]-y[9])**2)
        direction = np.arctan2(y[10]-y[9],x[10]-x[9])
        sin_dir[10] = np.sin(direction)
        cos_dir[10] = np.cos(direction)
        
        for t in range(1,10):
            
            speed[t] = np.sqrt((x[t+1]-x[t-1])**2+(y[t+1]-y[t-1])**2)/2
            direction = np.arctan2(y[t+1]-y[t-1],x[t+1]-x[t-1])
            sin_dir[t] = np.sin(direction)
            cos_dir[t] = np.cos(direction)
            
            
        data[' speed%d' % i] = speed
        data[' sin(dir)%d' % i] = sin_dir
        data[' cos(dir)%d' % i] = cos_dir
        
    return data



def acceleration(data):
    
    for i in range(10):
        
        a = np.zeros(11)
        
        speed = data[' speed%d' % i]
        
        a[0] = speed[1]-speed[0]
        a[10] = speed[10]-speed[9]
        
        for t in range(1,10):
            a[t] = (speed[t+1]-speed[t-1])/2
            
            
        data[' acceleration%d' % i] = a
        
    return data




def turning(data):
    
    for i in range(10):
        
        turn = np.zeros(11)
        
        sin_dir = data[' sin(dir)%d' % i]
        cos_dir = data[' cos(dir)%d' % i]
        direction = np.arctan2(sin_dir, cos_dir)
        direction = continuous_angle(direction)
        
        turn[0] = direction[1]-direction[0]
        turn[10] = direction[10]-direction[9]
        
        for t in range(1,10):
            turn[t] = (direction[t+1]-direction[t-1])/2
            
            
        data[' turning%d' % i] = turn
        
    return data

# add_features needs to be implemented
def add_features(data):    
    data = speed_direction(data)
    data = acceleration(data)
    data = turning(data)
    
    return data

# Reading and Processing 

in this part and next part we are going to iterate over all csv files and perform the following operation on each file:
<ol>
  <li>read the dataset</li>
  <li>clean the dataset</li>
  <li>add new features</li>
  <li>reshape the dataset to a (1,x) shaped numpy array</li>
  <li>merge all produced numpy arrays into a single matrix</li>
</ol>

final update: instead of adding features and data cleaning part we just get the agent data using get_agent function.

In [451]:
def read_data(path):
    data = pd.read_csv(path)
    return data

def get_path_X_train(i):
    string = '../data2/train/X/X_'+str(i)+'.csv'
    return string

def get_path_y_train(i):
    string = '../data2/train/y/y_'+str(i)+'.csv'
    return string

def get_path_X_val(i):
    string = '../data2/val/X/X_'+str(i)+'.csv'
    return string

def get_path_y_val(i):
    string = '../data2/val/y/y_'+str(i)+'.csv'
    return string

def get_path_X_test(i):
    string = '../data2/test/X/X_'+str(i)+'.csv'
    return string

def convert(data):
    return data.values.ravel()

def get_data_X_train(i):    
#     return convert(add_features(clean_data_X(read_data(get_path_X_train(i)))))
#     return convert(clean_data_X(read_data(get_path_X_train(i))))
    return convert(get_agent(read_data(get_path_X_train(i))))
#     return convert(get_agent_x(read_data(get_path_X_train(i)))), convert(get_agent_y(read_data(get_path_X_train(i))))

def get_data_y_train(i):
    return convert(clean_data_y(read_data(get_path_y_train(i))))

def get_data_X_val(i):
#     return convert(add_features(clean_data_X(read_data(get_path_X_val(i)))))
    return convert(get_agent(read_data(get_path_X_val(i))))
#     return convert(get_agent_x(read_data(get_path_X_val(i)))), convert(get_agent_y(read_data(get_path_X_val(i))))
def get_data_y_val(i):
    return convert(clean_data_y(read_data(get_path_y_val(i))))

def get_data_X_test(i):
#     return convert(add_features(clean_data_X(read_data(get_path_X_test(i)))))
    return convert(get_agent(read_data(get_path_X_test(i))))
#     return convert(get_agent_x(read_data(get_path_X_test(i)))), convert(get_agent_y(read_data(get_path_X_test(i))))


# Final Dataset

In [455]:
def get_data_train():
    n = 2308
    X_train = get_data_X_train(0).reshape(1,-1)
    y_train = get_data_y_train(0).reshape(1,-1)
    for i in range(1,n):
        X_train = np.concatenate((X_train,get_data_X_train(i).reshape(1,-1)))
        y_train = np.concatenate((y_train,get_data_y_train(i).reshape(1,-1)))
    return X_train, y_train

def get_data_val():
    n = 524
    X_val = get_data_X_val(0).reshape(1,-1)
    y_val = get_data_y_val(0).reshape(1,-1)
    for i in range(1,n):
        X_val = np.concatenate((X_val,get_data_X_val(i).reshape(1,-1)))
        y_val = np.concatenate((y_val,get_data_y_val(i).reshape(1,-1)))
    return X_val, y_val

def get_data_test():
    n = 20
    X_test = get_data_X_test(0).reshape(1,-1)
    for i in range(1,n):
        X_test = np.concatenate((X_test, get_data_X_test(i).reshape(1,-1)))
    return X_test

In [473]:
# def get_data_train():
#     n = 2308
#     X_train_x = get_data_X_train(0)[0].reshape(1,-1)
#     X_train_y = get_data_X_train(0)[1].reshape(1,-1)
#     y_train = get_data_y_train(0).reshape(1,-1)
#     for i in range(1,n):
#         X_train_x = np.concatenate((X_train_x,get_data_X_train(i)[0].reshape(1,-1)))
#         X_train_y = np.concatenate((X_train_y,get_data_X_train(i)[1].reshape(1,-1)))
#         y_train = np.concatenate((y_train,get_data_y_train(i).reshape(1,-1)))
#     return X_train_x, X_train_y, y_train

# def get_data_val():
#     n = 524
#     X_val_x = get_data_X_val(0)[0].reshape(1,-1)
#     X_val_y = get_data_X_val(0)[1].reshape(1,-1)
#     y_val = get_data_y_val(0).reshape(1,-1)
#     for i in range(1,n):
#         X_val_x = np.concatenate((X_val_x,get_data_X_val(i)[0].reshape(1,-1)))
#         X_val_y = np.concatenate((X_val_y,get_data_X_val(i)[1].reshape(1,-1)))
#         y_val = np.concatenate((y_val,get_data_y_val(i).reshape(1,-1)))
#     return X_val_x, X_val_y, y_val

# def get_data_test():
#     n = 20
#     X_test_x = get_data_X_test(0)[0].reshape(1,-1)
#     X_test_y = get_data_X_test(0)[1].reshape(1,-1)
#     for i in range(1,n):
#         X_test_x = np.concatenate((X_test_x,get_data_X_test(i)[0].reshape(1,-1)))
#         X_test_y = np.concatenate((X_test_y,get_data_X_test(i)[1].reshape(1,-1)))
#     return X_test_x, X_test_y

In [6]:
# X_train, y_train = get_data_train()
# X_val, y_val = get_data_val()
# X_test = get_data_test()

In [29]:
# X1_train, y1_train = get_data_train()
# X1_val, y1_val = get_data_val()
# X1_test = get_data_test()

In [430]:
X2_train, y2_train = get_data_train()
X2_val, y2_val = get_data_val()
X2_test = get_data_test()

In [474]:
# X3_train_x, X3_train_y, y2_train = get_data_train()
# X3_val_x, X3_val_y, y2_val = get_data_val()
# X3_test_x, X3_test_y = get_data_test()

# Saving The Processed Dataset

In [271]:
# pd.DataFrame(X_train).to_csv("../processed_data/X_train.csv")
# pd.DataFrame(y_train).to_csv("../processed_data/y_train.csv")
# pd.DataFrame(X_val).to_csv("../processed_data/X_val.csv")
# pd.DataFrame(y_val).to_csv("../processed_data/y_val.csv")
# pd.DataFrame(X_test).to_csv("../processed_data/X_test.csv")

In [404]:
pd.DataFrame(X2_train).to_csv("../processed_data2/X_train.csv")
pd.DataFrame(y2_train).to_csv("../processed_data2/y_train.csv")
pd.DataFrame(X2_val).to_csv("../processed_data2/X_val.csv")
pd.DataFrame(y2_val).to_csv("../processed_data2/y_val.csv")
pd.DataFrame(X2_test).to_csv("../processed_data2/X_test.csv")

In [ ]:
def chained_Ridge(X_train, y_train, X_test):
    model_x = Ridge()
    model_x.fit(X_train, y_train[:,0])
    x_hat_train = model_x.predict(X_train)
    x_hat_test = model_x.predict(X_test)

    model_y = Ridge()
    model_y.fit(X_train, y_train[:,1])
    y_hat_train = model_y.predict(X_train)
    y_hat_test = model_y.predict(X_test)

    y_pred_train = np.concatenate((x_hat_train.reshape(-1,1), y_hat_train.reshape(-1,1)),axis=1)
    y_pred_test = np.concatenate((x_hat_test.reshape(-1,1), y_hat_test.reshape(-1,1)),axis=1)

    for i in range(2,60):
        XN_train = np.concatenate((X_train,y_pred_train[:,i-2].reshape(-1,1)),axis =1)
        XN_test = np.concatenate((X_test,y_pred_test[:,i-2].reshape(-1,1)), axis = 1)

        model = Ridge()
        model.fit(XN_train, y_train[:,i])
        hat_train = model.predict(XN_train)
        hat_test = model.predict(XN_test)

        y_pred_train = np.concatenate((y_pred_train, hat_train.reshape(-1,1)),axis=1)
        y_pred_test = np.concatenate((y_pred_test, hat_test.reshape(-1,1)),axis=1)
        
    return y_pred_train, y_pred_test

In [ ]:
X3_train = np.concatenate((X2_train,X2_val),axis=0)
y3_train = np.concatenate((y2_train,y2_val),axis=0)
y_pred_train, y_pred_test = chained_KernelRidge(X3_train,y3_train,X2_test)

In [ ]:
filename = 'chained_NN_ONE_Layer_100N_s60_NotNormalized_NoRegularization.sav'

# y_pred = model.predict(X2_test)

xs = [str(i) + "_x_" + str(j) for i in range(20) for j in range(1, 31)]
ys = [str(i) + "_y_" + str(j) for i in range(20) for j in range(1, 31)]

ids = []
for i in range(len(xs)):
    ids.append(xs[i])
    ids.append(ys[i])

data = {'Id': ids,
        'location': y_pred_test.reshape(-1,1).flatten()}

df = pd.DataFrame(data, columns=['Id', 'location'])
df.to_csv('ytest.csv', index=False)

# Extra

In [11]:
# from sklearn.decomposition import PCA
# from sklearn.manifold import TSNE
# pca = PCA(n_components=3)
# pca_result = pca.fit_transform(X_train)
# df = pd.DataFrame((pca_result[:,0], pca_result[:,1], pca_result[:,2]))
# print('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_))

Explained variation per principal component: [0.17160224 0.08648861 0.07963303]


In [12]:
# tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
# tsne_results = tsne.fit_transform(X_train)

[t-SNE] Computing 121 nearest neighbors...
[t-SNE] Indexed 2307 samples in 0.862s...
[t-SNE] Computed neighbors for 2307 samples in 10.335s...
[t-SNE] Computed conditional probabilities for sample 1000 / 2307
[t-SNE] Computed conditional probabilities for sample 2000 / 2307
[t-SNE] Computed conditional probabilities for sample 2307 / 2307
[t-SNE] Mean sigma: 75.775670
[t-SNE] KL divergence after 250 iterations with early exaggeration: 75.650597
[t-SNE] KL divergence after 300 iterations: 2.322378


In [14]:
# tsne_results
# model = regre

(2307, 2)

In [18]:
# from sklearn.preprocessing import StandardScaler
# sc_X = StandardScaler()
# sc_y = StandardScaler()
# SX_train = sc_X.fit_transform(X_train)
# Sy_train = sc_y.fit_transform(y_train)

In [349]:
# from sklearn.kernel_ridge import KernelRidge
# from sklearn.multioutput import MultiOutputRegressor
# model = MultiOutputRegressor(KernelRidge(alpha=1.0))
# model.fit(X2_train, y_train)

MultiOutputRegressor(estimator=KernelRidge(alpha=1.0))

In [351]:
# from sklearn.svm import SVR
# model = MultiOutputRegressor(SVR(kernel='rbf'))
# model.fit(X2_train, y_train)

MultiOutputRegressor(estimator=SVR())

In [353]:
# from sklearn.linear_model import Ridge
# model = MultiOutputRegressor(Ridge())
# model.fit(X2_train, y_train)

MultiOutputRegressor(estimator=Ridge())

In [30]:
# from sklearn.linear_model import Ridge
# model = MultiOutputRegressor(Ridge(alpha=1.0))
# model.fit(X1_train, y1_train)

MultiOutputRegressor(estimator=Ridge())

In [347]:
# from sklearn import linear_model
# model = MultiOutputRegressor(linear_model.Lasso())
# model.fit(X2_train, y_train)

MultiOutputRegressor(estimator=Lasso())

In [362]:
# from sklearn.neural_network import MLPRegressor
# model = MultiOutputRegressor(MLPRegressor())
# model.fit(X2_train, y_train)

MultiOutputRegressor(estimator=MLPRegressor())

In [363]:
# y_pred = model.predict(X2_train)
# train_error = np.mean(np.sqrt(np.sum((y_pred - y_train)**2,axis=1)))
# print("train error: %f" %train_error)

# y_pred = model.predict(X2_val)
# validation_error = np.mean(np.sqrt(np.sum((y_pred - y_val)**2,axis=1)))
# print("validation error: %f" %validation_error)

train error: 4.910352
validation error: 4.768275


In [359]:
# pca = PCA(n_components=2)
# pca_results_train = pca.fit_transform(X2_train)
# pca_results_val = pca.fit_transform(X2_val)

In [360]:
# model = MultiOutputRegressor(Ridge(alpha=1))
# model.fit(pca_results_train, y_train)

MultiOutputRegressor(estimator=Ridge(alpha=1))

In [361]:
# y_pred = model.predict(pca_results_train)
# train_error = np.mean(np.sqrt(np.sum((y_pred - y_train)**2,axis=1)))
# print("train error: %f" %train_error)

# y_pred = model.predict(pca_results_val)
# validation_error = np.mean(np.sqrt(np.sum((y_pred - y_val)**2,axis=1)))
# print("validation error: %f" %validation_error)

train error: 7.369057
validation error: 28.320272


In [58]:
# from sklearn.svm import SVR
# model = MultiOutputRegressor(SVR(kernel='rbf'))
# model.fit(pca_results_train, y1_train)

MultiOutputRegressor(estimator=SVR())

In [69]:
# from sklearn.neural_network import MLPRegressor, MLPClassifier
# model = MultiOutputRegressor(MLPRegressor())
# model.fit(pca_results_train, y1_train)

MultiOutputRegressor(estimator=MLPRegressor())

In [84]:
# from sklearn import linear_model
# model = MultiOutputRegressor(linear_model.Lasso(alpha=0.001))
# model.fit(X1_train, y1_train)

MultiOutputRegressor(estimator=Lasso(alpha=0.001))

In [223]:
# k = 0
# y_xk_train = y1_train[:,k]
# y_xk_val = y1_val[:,k]
# XN_train = np.concatenate((X1_train,y1_train[:,k-2].reshape(-1,1)),axis =1)
# XN_val = np.concatenate((X1_val,y1_val[:,k-2].reshape(-1,1)), axis = 1)

In [224]:
# model = linear_model.Lasso()
# model.fit(XN_train, y_xk_train)

Lasso()

In [225]:
# model = linear_model.Lasso()
# model.fit(X1_train, y_xk_train)

Lasso()

In [226]:
# y_xk_pred = model.predict(X1_train)
# train_error = np.sqrt(np.mean((y_xk_pred - y1_train[:,k])**2))
# print("train error: %f" %train_error)

# y_xk_pred = model.predict(X1_val)
# validation_error = np.sqrt(np.mean((y_xk_pred - y1_val[:,k])**2))
# print("validation error: %f" %validation_error)

train error: 0.234284
validation error: 0.241532


In [217]:
# y_xk_pred = model.predict(XN_train)
# train_error = np.sqrt(np.mean((y_xk_pred - y1_train[:,k])**2))
# print("train error: %f" %train_error)

# y_xk_pred = model.predict(XN_val)
# validation_error = np.sqrt(np.mean((y_xk_pred - y1_val[:,k])**2))
# print("validation error: %f" %validation_error)

train error: 1.079208
validation error: 0.906598


In [220]:
# y_x0_pred = model.predict(X1_train)
# train_error = np.sqrt(np.mean((y_x0_pred - y1_train[:,k])**2))
# print("train error: %f" %train_error)

# y_x0_pred = model.predict(X1_val)
# validation_error = np.sqrt(np.mean((y_x0_pred - y1_val[:,k])**2))
# print("validation error: %f" %validation_error)

train error: 1.525991
validation error: 1.244259


In [74]:
# y_pred = model.predict(pca_results_train)
# train_error = np.mean(np.sqrt(np.sum((y_pred - y1_train)**2,axis=1)))
# print("train error: %f" %train_error)

# y_pred = model.predict(pca_results_val)
# validation_error = np.mean(np.sqrt(np.sum((y_pred - y1_val)**2,axis=1)))
# print("validation error: %f" %validation_error)

train error: 14.229291
validation error: 14.956994


In [85]:
# y_pred = model.predict(X1_train)
# train_error = np.mean(np.sqrt(np.sum((y_pred - y1_train)**2,axis=1)))
# print("train error: %f" %train_error)

# y_pred = model.predict(X1_val)
# validation_error = np.mean(np.sqrt(np.sum((y_pred - y1_val)**2,axis=1)))
# print("validation error: %f" %validation_error)

train error: 12.473731
validation error: 14.619175


In [431]:
def chained_Ridge(X_train, y_train, X_test):
    model_x = Ridge()
    model_x.fit(X_train, y_train[:,0])
    x_hat_train = model_x.predict(X_train)
    x_hat_test = model_x.predict(X_test)

    model_y = Ridge()
    model_y.fit(X_train, y_train[:,1])
    y_hat_train = model_y.predict(X_train)
    y_hat_test = model_y.predict(X_test)

    y_pred_train = np.concatenate((x_hat_train.reshape(-1,1), y_hat_train.reshape(-1,1)),axis=1)
    y_pred_test = np.concatenate((x_hat_test.reshape(-1,1), y_hat_test.reshape(-1,1)),axis=1)

    for i in range(2,60):
        XN_train = np.concatenate((X_train,y_pred_train[:,i-2].reshape(-1,1)),axis =1)
        XN_test = np.concatenate((X_test,y_pred_test[:,i-2].reshape(-1,1)), axis = 1)

        model = Ridge()
        model.fit(XN_train, y_train[:,i])
        hat_train = model.predict(XN_train)
        hat_test = model.predict(XN_test)

        y_pred_train = np.concatenate((y_pred_train, hat_train.reshape(-1,1)),axis=1)
        y_pred_test = np.concatenate((y_pred_test, hat_test.reshape(-1,1)),axis=1)
        
    return y_pred_train, y_pred_test

In [445]:
def chained_Ridge_beta(X_train, y_train, X_test):
    model_x = Ridge()
    model_x.fit(X_train, y_train[:,0])
    x_hat_train = model_x.predict(X_train)
    x_hat_test = model_x.predict(X_test)

    model_y = Ridge()
    model_y.fit(X_train, y_train[:,1])
    y_hat_train = model_y.predict(X_train)
    y_hat_test = model_y.predict(X_test)

    y_pred_train = np.concatenate((x_hat_train.reshape(-1,1), y_hat_train.reshape(-1,1)),axis=1)
    y_pred_test = np.concatenate((x_hat_test.reshape(-1,1), y_hat_test.reshape(-1,1)),axis=1)
    Xx_train = X_train
    Xx_test = X_test
    Xy_train = X_train
    Xy_test = X_test
    for i in range(1,30):
        kx = 2*i
        ky = 2*i+1
        
        Xx_train = np.concatenate((Xx_train,y_pred_train[:,kx-2].reshape(-1,1)),axis =1)
        Xx_test = np.concatenate((Xx_test,y_pred_test[:,kx-2].reshape(-1,1)), axis = 1)
        
        Xy_train = np.concatenate((Xy_train,y_pred_train[:,ky-2].reshape(-1,1)),axis =1)
        Xy_test = np.concatenate((Xy_test,y_pred_test[:,ky-2].reshape(-1,1)), axis = 1)
        

#         model = Ridge()
#         model.fit(XN_train, y_train[:,i])
#         hat_train = model.predict(XN_train)
#         hat_test = model.predict(XN_test)

#         y_pred_train = np.concatenate((y_pred_train, hat_train.reshape(-1,1)),axis=1)
#         y_pred_test = np.concatenate((y_pred_test, hat_test.reshape(-1,1)),axis=1)
        model_x = Ridge()
        model_x.fit(Xx_train, y_train[:,0])
        x_hat_train = model_x.predict(Xx_train)
        x_hat_test = model_x.predict(Xx_test)

        model_y = Ridge()
        model_y.fit(Xy_train, y_train[:,1])
        y_hat_train = model_y.predict(Xy_train)
        y_hat_test = model_y.predict(Xy_test)

        y_pred_train = np.concatenate((y_pred_train, x_hat_train.reshape(-1,1), y_hat_train.reshape(-1,1)),axis=1)
        y_pred_test = np.concatenate((y_pred_test, x_hat_test.reshape(-1,1), y_hat_test.reshape(-1,1)),axis=1)
        
    return y_pred_train, y_pred_test

In [653]:
def chained_Ridge_x(X_train, y_train, X_test):
    alpha = 1
    model_x = Ridge(alpha)
    model_x.fit(X_train, y_train[:,0])
    x_hat_train = model_x.predict(X_train)
    x_hat_test = model_x.predict(X_test)


    y_pred_train_x = x_hat_train.reshape(-1,1)
    y_pred_test_x = x_hat_test.reshape(-1,1)
    
    XN_train = X_train
    XN_test = X_test
    for i in range(1,30):        
        XN_train = np.concatenate((XN_train,y_pred_train_x[:,i-1].reshape(-1,1)),axis =1)
        XN_test = np.concatenate((XN_test,y_pred_test_x[:,i-1].reshape(-1,1)), axis = 1)

        model = Ridge(alpha)
        k = 6
        model.fit(XN_train[:,-k:-1], y_train[:,2*i])
        hat_train = model.predict(XN_train[:,-k:-1])
        hat_test = model.predict(XN_test[:,-k:-1])

        y_pred_train_x = np.concatenate((y_pred_train_x, hat_train.reshape(-1,1)),axis=1)
        y_pred_test_x = np.concatenate((y_pred_test_x, hat_test.reshape(-1,1)),axis=1)
        
    return y_pred_train_x, y_pred_test_x

In [654]:
X4_train_x = np.concatenate((X3_train_x,X3_val_x),axis=0)
y4_train = np.concatenate((y2_train,y2_val),axis=0)
y_pred_train_x, y_pred_test_x = chained_Ridge_x(X4_train_x,y4_train,X3_test_x)

k=29
train_error = np.sqrt(np.mean((y_pred_train_x[:,k] - y4_train[:,2*k])**2))
print("train error: %f" %train_error)

train error: 2.745274


In [585]:
model_x = Ridge()
model_x.fit(X3_train_x, y2_train[:,0])
x_hat_train = model_x.predict(X3_train_x)
x_hat_test = model_x.predict(X3_test_x)
y_pred_train_x = x_hat_train.reshape(-1,1)
y_pred_test_x = x_hat_test.reshape(-1,1)
XN_train = X3_train_x
XN_test = X3_test_x
for i in range(1,3):
    XN_train = np.concatenate((XN_train,y_pred_train_x[:,i-1].reshape(-1,1)),axis=1)
    XN_test = np.concatenate((XN_test,y_pred_test_x[:,i-1].reshape(-1,1)), axis = 1)
    
    model = Ridge()
    model.fit(XN_train, y2_train[:,2*i])
    hat_train = model.predict(XN_train)
    hat_test = model.predict(XN_test)

    y_pred_train_x = np.concatenate((y_pred_train_x, hat_train.reshape(-1,1)),axis=1)
    y_pred_test_x = np.concatenate((y_pred_test_x, hat_test.reshape(-1,1)),axis=1)

In [586]:
y_pred_test_x.shape

(20, 3)

In [512]:
model = MultiOutputRegressor(Ridge(alpha=1))
model.fit(X3_train_x, y2_train[:,2*np.arange(30)])

MultiOutputRegressor(estimator=Ridge(alpha=1))

In [513]:
k = 29
y_x0_pred = model.predict(X3_train_x)
train_error = np.sqrt(np.mean((y_x0_pred[:,k] - y2_train[:,2*k])**2))
print("train error: %f" %train_error)

y_x0_pred = model.predict(X3_val_x)
validation_error = np.sqrt(np.mean((y_x0_pred[:,k] - y2_val[:,2*k])**2))
print("validation error: %f" %validation_error)

train error: 3.339322
validation error: 3.280116


In [514]:
model = MultiOutputRegressor(Ridge(alpha=1))
model.fit(X3_train_y, y2_train[:,2*np.arange(30)+1])

MultiOutputRegressor(estimator=Ridge(alpha=1))

In [516]:
k = 29
y_x0_pred = model.predict(X3_train_y)
train_error = np.sqrt(np.mean((y_x0_pred[:,k] - y2_train[:,2*k+1])**2))
print("train error: %f" %train_error)

y_x0_pred = model.predict(X3_val_y)
validation_error = np.sqrt(np.mean((y_x0_pred[:,k] - y2_val[:,2*k+1])**2))
print("validation error: %f" %validation_error)

train error: 13.112452
validation error: 13.148263


In [537]:
y_pred, y_val = chained_Ridge(X2_train, y2_train, X2_val)
k = 58

# y_x0_pred = model.predict(X3_train_y)
train_error = np.sqrt(np.mean((y_pred[:,k] - y2_train[:,k])**2))
print("train error: %f" %train_error)

# y_x0_pred = model.predict(X3_val_y)
validation_error = np.sqrt(np.mean((y_val[:,k] - y2_val[:,k])**2))
print("validation error: %f" %validation_error)

train error: 2.745007
validation error: 2.735654


In [538]:
model = MultiOutputRegressor(Ridge(alpha=1))
model.fit(X2_train, y2_train)
k = 58

y_pred = model.predict(X2_train)
train_error = np.sqrt(np.mean((y_pred[:,k] - y2_train[:,k])**2))
print("train error: %f" %train_error)

y_val = model.predict(X2_val)
validation_error = np.sqrt(np.mean((y_val[:,k] - y2_val[:,k])**2))
print("validation error: %f" %validation_error)

train error: 3.231556
validation error: 3.200428


In [606]:
X4_train_x = np.concatenate((X3_train_x,X3_val_x),axis=0)
y4_train = np.concatenate((y2_train,y2_val),axis=0)
y_pred_train_x, y_pred_test_x = chained_Ridge_x(X4_train_x,y4_train,X3_test_x)

k=29
train_error = np.sqrt(np.mean((y_pred_train_x[:,k] - y4_train[:,2*k])**2))
print("train error: %f" %train_error)

train error: 3.749028


In [439]:
def chained_KernelRidge(X_train, y_train, X_test):
    model_x = SVR(kernel='rbf')
    model_x.fit(X_train, y_train[:,0])
    x_hat_train = model_x.predict(X_train)
    x_hat_test = model_x.predict(X_test)

    model_y = SVR(kernel='rbf')
    model_y.fit(X_train, y_train[:,1])
    y_hat_train = model_y.predict(X_train)
    y_hat_test = model_y.predict(X_test)

    y_pred_train = np.concatenate((x_hat_train.reshape(-1,1), y_hat_train.reshape(-1,1)),axis=1)
    y_pred_test = np.concatenate((x_hat_test.reshape(-1,1), y_hat_test.reshape(-1,1)),axis=1)

    for i in range(2,60):
        XN_train = np.concatenate((X_train,y_pred_train[:,i-2].reshape(-1,1)),axis =1)
        XN_test = np.concatenate((X_test,y_pred_test[:,i-2].reshape(-1,1)), axis = 1)

        model = KernelRidge()
        model.fit(XN_train, y_train[:,i])
        hat_train = model.predict(XN_train)
        hat_test = model.predict(XN_test)

        y_pred_train = np.concatenate((y_pred_train, hat_train.reshape(-1,1)),axis=1)
        y_pred_test = np.concatenate((y_pred_test, hat_test.reshape(-1,1)),axis=1)
        
    return y_pred_train, y_pred_test

In [446]:
X3_train = np.concatenate((X2_train,X2_val),axis=0)
y3_train = np.concatenate((y2_train,y2_val),axis=0)
y_pred_train, y_pred_test = chained_Ridge(X3_train,y3_train,X2_test)

In [ ]:
filename = 'chained_NN_ONE_Layer_100N_s60_NotNormalized_NoRegularization.sav'

# y_pred = model.predict(X2_test)

xs = [str(i) + "_x_" + str(j) for i in range(20) for j in range(1, 31)]
ys = [str(i) + "_y_" + str(j) for i in range(20) for j in range(1, 31)]

ids = []
for i in range(len(xs)):
    ids.append(xs[i])
    ids.append(ys[i])

data = {'Id': ids,
        'location': y_pred_test.reshape(-1,1).flatten()}

df = pd.DataFrame(data, columns=['Id', 'location'])
df.to_csv('ytest.csv', index=False)

# Extra

In [448]:
# y_pred = model.predict(X1_train)
train_error = np.mean(np.sqrt(np.sum((y_pred_train - y3_train)**2,axis=1)))
print("train error: %f" %train_error)

# y_pred = model.predict(X1_val)
# validation_error = np.mean(np.sqrt(np.sum((y_pred_test - y2_val)**2,axis=1)))
# print("validation error: %f" %validation_error)

train error: 3.595748


In [442]:
X3_train = np.concatenate((X2_train,X2_val),axis=0)
y3_train = np.concatenate((y2_train,y2_val),axis=0)
y_pred_train, y_pred_test = chained_KernelRidge(X3_train,y3_train,X2_test)
# y_pred_train, y_pred_test = chained_Ridge(X2_train,y2_train,X2_val)
# test_error = []
# for i in range(60):
#     test_error.append(np.sqrt(np.mean((y_pred_test[:,i] - y_test[:,i])**2)))

In [443]:
train_error = np.mean(np.sqrt(np.sum((y_pred_train - y3_train)**2,axis=1)))
print("train error: %f" %train_error)

train error: 5.290451


In [415]:
from sklearn.linear_model import Ridge
model = MultiOutputRegressor(Ridge())
model.fit(X3_train, y3_train)
y_pred_train = model.predict(X3_train)

In [421]:
# y_pred = model.predict(X1_train)
y_pred_train = model.predict(X3_train)
train_error = np.mean(np.sqrt(np.sum((y_pred_train - y3_train)**2,axis=1)))
print("train error: %f" %train_error)


train error: 4.788315


In [426]:
model = NeuralNet_Chain(hidden_layer_sizes=[[100]],classification=False, lammy=np.array([0.001]),s=60, max_iter=np.array([1000]), verbose=True)
model.fit(X2_train,y2_train)

3 - loss: 516879.737
6 - loss: 498516.137
7 - loss: 490142.202
8 - loss: 483288.899
9 - loss: 458613.773
10 - loss: 417517.070
11 - loss: 362342.323
13 - loss: 354642.117
14 - loss: 349707.557
15 - loss: 346502.219
16 - loss: 326734.796
17 - loss: 305155.926
18 - loss: 297783.957
19 - loss: 292683.959
20 - loss: 290747.590
21 - loss: 289859.269
22 - loss: 288725.403
23 - loss: 283956.386
24 - loss: 281022.941
25 - loss: 280158.181
26 - loss: 279676.348
27 - loss: 279243.707
28 - loss: 275236.325
29 - loss: 268709.266
30 - loss: 265096.714
31 - loss: 263601.441
32 - loss: 262702.112
33 - loss: 262333.191
34 - loss: 262016.791
35 - loss: 260630.524
36 - loss: 248801.162
37 - loss: 215480.976
39 - loss: 214782.180
40 - loss: 214539.415
41 - loss: 214321.381
42 - loss: 213461.475
43 - loss: 212480.861
44 - loss: 210236.777
45 - loss: 209514.996
46 - loss: 209325.945
47 - loss: 209150.153
48 - loss: 208245.109
49 - loss: 138126.349
51 - loss: 128129.598
52 - loss: 128001.933
53 - loss: 1279

410 - loss: 56319.427
411 - loss: 56319.357
412 - loss: 56317.361
413 - loss: 56315.358
414 - loss: 56314.562
415 - loss: 56314.491
416 - loss: 56314.421
417 - loss: 56314.306
418 - loss: 56159.177
420 - loss: 56158.630
421 - loss: 56158.558
422 - loss: 56158.488
423 - loss: 56155.925
424 - loss: 56153.333
425 - loss: 56150.585
426 - loss: 56150.484
427 - loss: 56150.415
428 - loss: 56150.346
429 - loss: 56145.110
430 - loss: 56139.770
431 - loss: 56139.319
432 - loss: 56139.237
433 - loss: 56139.169
434 - loss: 56138.898
435 - loss: 56137.125
436 - loss: 56135.324
437 - loss: 56131.552
438 - loss: 56131.476
439 - loss: 56131.407
440 - loss: 56131.339
441 - loss: 56111.283
442 - loss: 56106.629
443 - loss: 56106.463
444 - loss: 56106.394
445 - loss: 56106.333
447 - loss: 55554.507
449 - loss: 55543.148
450 - loss: 55542.962
451 - loss: 55542.260
452 - loss: 55540.007
453 - loss: 55538.301
454 - loss: 55537.668
455 - loss: 55537.435
456 - loss: 55537.353
457 - loss: 55537.216
458 - loss

797 - loss: 51581.614
798 - loss: 51580.242
799 - loss: 51580.069
800 - loss: 51579.956
801 - loss: 51579.841
802 - loss: 51576.816
803 - loss: 51572.840
804 - loss: 51571.647
805 - loss: 51571.012
806 - loss: 51570.892
807 - loss: 51570.779
808 - loss: 51570.651
809 - loss: 51565.120
810 - loss: 51557.672
811 - loss: 51556.730
812 - loss: 51556.443
813 - loss: 51556.324
814 - loss: 51556.211
815 - loss: 51555.242
816 - loss: 51554.104
817 - loss: 51546.569
818 - loss: 51523.287
819 - loss: 51523.023
820 - loss: 51522.895
821 - loss: 51522.776
822 - loss: 51521.987
823 - loss: 51521.203
824 - loss: 51507.165
825 - loss: 51506.655
826 - loss: 51506.521
827 - loss: 51506.409
828 - loss: 51472.029
829 - loss: 51356.967
830 - loss: 51308.263
831 - loss: 51308.116
832 - loss: 51307.854
833 - loss: 51307.039
834 - loss: 51306.264
835 - loss: 51304.884
836 - loss: 51304.725
837 - loss: 51304.615
838 - loss: 51304.506
839 - loss: 51300.576
840 - loss: 51296.354
841 - loss: 51293.485
842 - loss

In [434]:
filename = 'chained_NN_ONE_Layer_100N_s60_NotNormalized_NoRegularization.sav'

# y_pred = model.predict(X2_test)

xs = [str(i) + "_x_" + str(j) for i in range(20) for j in range(1, 31)]
ys = [str(i) + "_y_" + str(j) for i in range(20) for j in range(1, 31)]

ids = []
for i in range(len(xs)):
    ids.append(xs[i])
    ids.append(ys[i])

data = {'Id': ids,
        'location': y_pred_test.reshape(-1,1).flatten()}

df = pd.DataFrame(data, columns=['Id', 'location'])
df.to_csv('ytest.csv', index=False)

In [423]:
df

,Id,location
0,0_x_1,1.020550
1,0_y_1,-0.002434
2,0_x_2,2.023388
3,0_y_2,-0.008287
4,0_x_3,3.042025
...,...,...
1195,19_y_28,-0.141414
1196,19_x_29,11.480890
1197,19_y_29,-0.149091
1198,19_x_30,13.159775


In [ ]:
X_train = pd.read_csv('../data/X_train.csv')
X_train = np.array(X_train.iloc[:,1:])
y_train = pd.read_csv('../data/y_train.csv')
y_train = np.array(y_train.iloc[:,1:])
X_val = pd.read_csv('../data/X_val.csv')
X_val = np.array(X_val.iloc[:,1:])
y_val = pd.read_csv('../data/y_val.csv')
y_val = np.array(y_val.iloc[:,1:])
X_test = pd.read_csv('../data/X_test.csv')
X_test = np.array(X_test.iloc[:,1:])

print("X_train  \n n = %d, d = %d" %(X_train.shape[0],X_train.shape[1]))
print("y_train  \n n = %d, d = %d" %(y_train.shape[0],y_train.shape[1]))
print("X_val  \n n = %d, d = %d" %(X_val.shape[0],X_val.shape[1]))
print("y_val  \n n = %d, d = %d" %(y_val.shape[0],y_val.shape[1]))
print("X_test  \n n = %d, d = %d" %(X_test.shape[0],X_test.shape[1]))

# X_train_normalized, mu_x, sigma_x = utils.standardize_cols(X_train)
# y_train_normalized, mu_y, sigma_y = utils.standardize_cols(y_train)
# X_val_normalized, _, _ = utils.standardize_cols(X_val,mu_x,sigma_x)
# y_val_normalized, _, _ = utils.standardize_cols(y_val,mu_y,sigma_y)
# X_test_normalized, _, _ = utils.standardize_cols(X_test,mu_x,sigma_x)
# model = NeuralNet_Chain_skLearn(hidden_layer_sizes=[(200,100)],classification=False, lammy=np.array([1]),s=60, max_iter=np.array([1000]), verbose=True, activation='relu')
model = NeuralNet_Chain(hidden_layer_sizes=[[100]],classification=False, lammy=np.array([0.001]),s=60, max_iter=np.array([1000]), verbose=True)
model.fit(np.concatenate((X_train,X_val),axis=0),np.concatenate((y_train,y_val),axis=0))

# model = MLPRegressor(alpha=1.0,verbose='True')
# model.fit(X_train, y_train)

# y_pred = model.predict(X_train_normalized)
# train_error = np.mean(np.sqrt(np.sum((y_pred - y_train)**2,axis=1)))
# print("train error: %f" %train_error)
#
# y_pred = model.predict(X_val_normalized)
# validation_error = np.mean(np.sqrt(np.sum((y_pred - y_val)**2,axis=1)))
# print("validation error: %f" %validation_error)